### Backend = tensorflow

In [3]:
%env KERAS_BACKEND = tensorflow

env: KERAS_BACKEND=tensorflow


### 匯入最基本套件

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact , FloatSlider

### 神經網路需要的套件

In [4]:
# Keras functions
from keras.models import Sequential , Model
from keras.layers import Input , Lambda , concatenate
from keras.layers import Dense
from keras.optimizers import Adam , SGD
from keras import metrics

# Keras dataset
from keras.datasets import mnist

# Keras utilis function
from keras.utils import np_utils

from keras import backend as K


Using TensorFlow backend.


### 讀入數據

In [5]:
(x_train0 , y_train0) , (x_test0 , y_test0) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [7]:
x_train = x_train0.reshape(60000,-1)
x_test = x_test0.reshape(10000,-1)

x_train -= x_train.min()
x_train = x_train/x_train.max()
x_test -= x_test.min()
x_test = x_test/x_test.max()

### VAE的建立

In [10]:
enc_1 = Dense(100, activation = 'sigmoid')

enc_mean = Dense(2)
enc_log_var = Dense(2)

dec_2 = Dense(100, activation = 'sigmoid')
dec_1 = Dense(784, activation = 'sigmoid')

In [12]:
x = Input(shape=(784,))
enc_x = enc_1(x)

z_mean = enc_mean(enc_x)
z_log_var = enc_log_var(enc_x)

### 建立sampling函數

In [13]:
def sampling(args):
    z_mean , z_log_var = args
    epsilon = K.random_normal(shape=(2,),mean=0. , stddev=1)
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [14]:
z = Lambda(sampling, output_shape=(2,))([z_mean, z_log_var])

In [16]:
dec_x = dec_2(z)
x_hat = dec_1(dec_x)

In [17]:
VAE = Model(x, x_hat)

In [18]:
VAE.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 100)          78500       input_1[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 2)            202         dense_5[0][0]                    
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 2)            202         dense_5[0][0]                    
__________________________________________________________________________________________________
lambda_1 (

In [19]:
def vae_loss(x, x_recon):  
    recovery_loss = 784 * metrics.binary_crossentropy(x, x_recon)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return recovery_loss + kl_loss

### 訓練

In [20]:
VAE.compile(loss=vae_loss, optimizer=Adam())

In [22]:
VAE.load_weights('VAE_handwriting_model_weights.h5')

### 子模型


In [25]:
VAE_Encoder = Model(x, z_mean)
VAE_Encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 202       
Total params: 78,702
Trainable params: 78,702
Non-trainable params: 0
_________________________________________________________________


In [27]:
h_input = Input(shape=(2,))
VAE_Decoder = Model(h_input, dec_1(dec_2(h_input)))
VAE_Decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               300       
_________________________________________________________________
dense_9 (Dense)              (None, 784)               79184     
Total params: 79,484
Trainable params: 79,484
Non-trainable params: 0
_________________________________________________________________


In [63]:
def inBetween(t):
    data_0 = x_train0[idx_1]
    data_1 = x_train0[idx_2]
    data_t = (1-t)*x_train0[idx_1] + t*x_train0[idx_2]

    mu_0 = VAE_Encoder.predict(x_train[idx_1:idx_1+1]).squeeze()
    mu_1 = VAE_Encoder.predict(x_train[idx_2:idx_2+1]).squeeze()
    mu_t = (1-t)*mu_0 + t*mu_1

    plt.figure(figsize=(10, 5))
    ax1 = plt.subplot(2, 1, 2)
    ax1.scatter(mu_0[0], mu_0[1])
    ax1.scatter(mu_1[0], mu_1[1])
    ax1.scatter(mu_t[0], mu_t[1])
    X = [mu_0[0],mu_1[0],mu_t[0]]
    Y = [mu_0[1],mu_1[1],mu_t[1]]
    ax1.plot(X, Y)
    n = ['(%d,%d) '% (mu_0[0], mu_0[1]),'(%d,%d) '% (mu_1[0], mu_1[1]),'(%d,%d) '% (mu_t[0], mu_t[1])]
    for i, txt in enumerate(n):
        ax1.annotate(txt, (X[i], Y[i]))

    ax2 = plt.subplot(2, 3, 1)
    ax2.imshow(data_0, cmap='Greys')

    ax3 = plt.subplot(2, 3, 2)
    ax3.imshow(data_t, cmap='Greys')

    ax4 = plt.subplot(2, 3, 3)
    ax4.imshow(data_1, cmap='Greys')

In [64]:
idx_1, idx_2 = np.random.randint(x_test.shape[0], size=2)
data_0 = x_train0[idx_1]
data_1 = x_train0[idx_2]

In [65]:
interact(inBetween, t=FloatSlider(value=0.5, 
                                  min=0, 
                                  max=1.0,
                                  step=0.02,))

interactive(children=(FloatSlider(value=0.5, description='t', max=1.0, step=0.02), Output()), _dom_classes=('w…

<function __main__.inBetween(t)>